In [ ]:
# Shift + Enter to run the current cell
print('Hello World')

# This is an example Marakdown cell
This is nice to write down instructions and format text with title, **bold** and _italic_

You can change the cell type in the top bar menu

# Datasets
#### finger 
link = https://www.bbci.de/competition/download/competition_iv/BCICIV_4_mat.zip
eval = https://www.bbci.de/competition/iv/results/ds4/true_labels.zip

#### foot tracking
download page: http://neurotycho.org/expdatalist/listview?task=36


# Part 0 : import libraries

In [ ]:
# Import packages

import numpy as np # library providing efficient array manipulation
import sklearn # machine learning tools
import scipy
import matplotlib.pyplot as plt # matlab-like plot library
import matplotlib
matplotlib.rcParams.update({'font.size': 22})

from librosa.util import frame
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, balanced_accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA

from scipy.signal import iirdesign, sosfilt, convolve, wiener

import tp # load python module written for this project

# Part 0 : Dataset

ECoG dataset of finger tapping, more details here: https://www.bbci.de/competition/iv/desc_4.pdf

### Download dataset


In [ ]:
dataset_path = 'data/bciciv'
if not tp.dataset_exists(dataset_path):
    tp.download_dataset(dataset_path)
else:
    print(f'dataset already exists in {dataset_path}') 


### Load dataset

In [ ]:
dataset = tp.load_data(dataset_path)
print(dataset)
fs = 1000

### Explore dataset

In [ ]:
for subject in dataset:
    print(subject)

In [ ]:
print(len(dataset))

In [ ]:
subject0 = dataset[0] # first subject in dataset
print(subject0.train_fingers)

In [ ]:
subject0.train_fingers.shape

In [ ]:
# get first sample for all fingers, indexing starts at 0 /!\
subject0.train_fingers[0, :]

In [ ]:
subject0.train_fingers[-1, :] # get last sample for all fingers

In [ ]:
subject0.train_fingers[1200:1300, 4] # get 100 samples from index 1200 to 1299 for the 5th finger

#### Plot the data with matplotlib
You can find the documentation for the `plot` function here: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html

For more information, you can check the [quick start guide](https://matplotlib.org/stable/users/explain/quick_start.html)

In [ ]:
n_fingers = subject0.train_fingers.shape[1]

fig = plt.figure(figsize=(15,10))
axes = fig.subplots(n_fingers, 1, sharex=True)

for i in range(n_fingers):
    axes[i].plot(subject0.train_fingers[:,i])
    axes[i].set_ylabel(f'finger {i+1}')

fig.suptitle('Finger trajectories (train set)')

In [ ]:
f_ratio = 40
subject_data = dataset[0]


(n_times_train, n_fingers) = subject_data.train_fingers.shape
n_channels = subject_data.train_ecog.shape[1]
n_times_train = n_times_train//f_ratio
threshold = 0.5
n_times_test = subject_data.test_fingers.shape[0]//f_ratio

win_size = 25
win = 1/win_size*np.ones([win_size, 1])
train_fingers_filtered = convolve(subject_data.train_fingers[::f_ratio], win, 'same')
train_fingers_filtered = np.concatenate((train_fingers_filtered, threshold*np.ones([n_times_train, 1])), axis=1)
y_train = np.argmax(train_fingers_filtered, axis=1)
cue_train = np.zeros([n_times_train, n_fingers+1])
for i in range(n_times_train):
    cue_train[i, y_train[i]] = 1

test_fingers_filtered = convolve(subject_data.test_fingers[::f_ratio], win, 'same')
test_fingers_filtered = np.concatenate((test_fingers_filtered, threshold*np.ones([n_times_test, 1])), axis=1)
y_test = np.argmax(test_fingers_filtered, axis=1)
cue_test = np.zeros([n_times_test, n_fingers+1])
for i in range(n_times_test):
    cue_test[i, y_test[i]] = 1

# Save the full data for future use
y_train_full = y_train
y_test_full = y_test

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)
for i in range(n_fingers):
    ax[0].plot(subject_data.train_fingers[::f_ratio, i])
    ax[1].plot(train_fingers_filtered[:, i])
    ax[2].plot(cue_train[:, i])
ax[1].plot([0, n_times_train], [threshold, threshold], color='k')
fig.suptitle('Finger trajectories (train set)')


fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)
for i in range(n_fingers):
    ax[0].plot(subject_data.test_fingers[::f_ratio, i])
    ax[1].plot(test_fingers_filtered[:, i])
    ax[2].plot(cue_test[:, i])
ax[1].plot([0, n_times_test], [threshold, threshold], color='k')
fig.suptitle('Finger trajectories (test set)')

# click on the figure to expand

In [ ]:
fig = plt.figure(figsize=(50,5))
ax = fig.subplots()

ax.plot(subject_data.train_ecog[:, 0])

fig.suptitle('First ecog channel (train set)')

# Part 1 : Dicrete decoding

In this part we will be interested in decoding finger mouvement during tapping from ECOG data and in order to do so we will focus on start with feature extraction by extracting time-frequency features on every electrodes using short term fourier transfrom. 

After this we will get a first set of results that we will visualize using different metrics such as the confusion matrix, the accuracy and the balanced accuracy. 

Then we will have an example of the usefullness of a validation set in the case of lagged responses. 

Once the lag have been set, we will see that we can also get better results using spatial filters such as PCA.

## 1.1 Feature extraction

### Temporal filtering

In this part we will focus on the filtering of the data using classical filtering function such as `iirdesign` and `sosfilt` from `scipy.signal` (https://docs.scipy.org/doc/scipy/reference/signal.html). 
It is pretty common in ECOG to filter in several frequency bands and use all the resulting features for the decoding.

In [ ]:
# bands = [(1, 60), (60, 100), (100, 200)]
# bands = [(1, 10)]
# for i in range(1, 20):
#     bands.append((i*10, (i+1)*10))
filters = []

bands = [(1, 10), (10,30), (30,50), (70,200)]
n_filters = len(bands)

for i_band, band in enumerate(bands):
   # filters.append(iirdesign(band[0], band[1], 1, 60, analog=False, fs=fs, output='sos'))
    filters.append(iirdesign(band, (band[0]*0.9, band[1]*1.1), 1, 60, analog=False, fs=fs, output='sos'))

scaler = StandardScaler()
train_ecog_normalized = scaler.fit_transform(subject_data.train_ecog)
test_ecog_normalized = scaler.fit_transform(subject_data.test_ecog)

train_ecog_filtered = []
test_ecog_filtered = []

for filt in filters:
    train_ecog_filtered.append(sosfilt(filt, train_ecog_normalized, axis=0))
    test_ecog_filtered.append(sosfilt(filt, test_ecog_normalized, axis=0))

fig = plt.figure(figsize=(50,15))
ax = fig.subplots(nrows=4, ncols=1)

ax[0].plot(train_ecog_filtered[0][:1000, 0])
ax[1].plot(train_ecog_filtered[1][:1000, 0])
ax[2].plot(train_ecog_filtered[2][:1000, 0])
ax[3].plot(train_ecog_filtered[3][:1000, 0])

ax[0].plot(train_ecog_normalized[:1000, 0])
ax[1].plot(train_ecog_normalized[:1000, 0])
ax[2].plot(train_ecog_normalized[:1000, 0])
ax[3].plot(train_ecog_normalized[:1000, 0])

fig.suptitle('Filtered data (train set)')

### Average power computation

Once the signal has been filtered, you can extract the average power of the signal in those frequency band using the `power` and `mean` function from `numpy`. Note that the formula for the average power of a centered signal $x$ is 

\begin{align}
P(x) = \frac{1}{N}\sum_{t}x^2(t),
\end{align}

with $N$ the number of elements in the window.  

In this part we will extract power in 1s of signal every 40ms. In order to compute the frame to be averaged you can use the fonction `frame` from `librosa.util`(https://librosa.org/doc/latest/generated/librosa.util.frame.html) and the padding function `pad` from `numpy` because for the first frames you can't take the 1s of previous signal as it doesn't exist, therefore you can replace it with zeros.  

In [ ]:
win_size = 1000
wiener_win_size = 25
n_buffer = 1;

X_train_1kHz = np.pad(np.power(np.concatenate(train_ecog_filtered, axis=1), 2), [(win_size-f_ratio, 0), (0, 0)])
X_train = np.mean(frame(X_train_1kHz, frame_length=win_size, hop_length=f_ratio, axis=0),axis=1)
X_train = frame(np.pad(X_train, [(n_buffer-1, 0), (0, 0)]), frame_length=n_buffer, hop_length=1, axis=0).reshape(n_times_train, n_buffer*n_channels*n_filters)
# X_train = wiener(X_train, (1, wiener_win_size))
X_train_full = X_train

X_test_1kHz = np.pad(np.power(np.concatenate(test_ecog_filtered, axis=1), 2), [(win_size-f_ratio, 0), (0, 0)])
X_test = np.mean(frame(X_test_1kHz, frame_length=win_size, hop_length=f_ratio, axis=0),axis=1)
X_test = frame(np.pad(X_test, [(n_buffer-1, 0), (0, 0)]), frame_length=n_buffer, hop_length=1, axis=0).reshape(n_times_test, n_buffer*n_channels*n_filters)

# X_test = wiener(X_test, (1, wiener_win_size))
X_test_full = X_test

### Set if we use the idle or not

In order to make things easier you can chose to select only the fingers and not the idle task.

In [ ]:

use_idle_data = True
if not use_idle_data:
    X_train = X_train_full[y_train_full != 5]
    X_test = X_test_full[y_test_full != 5]
    y_train = y_train_full[y_train_full != 5]
    y_test = y_test_full[y_test_full != 5]
    n_times_train = y_train.shape[0]
    n_times_test = y_test.shape[0]
else:
    X_train = X_train_full
    X_test = X_test_full
    y_train = y_train_full
    y_test = y_test_full
    n_times_train = y_train.shape[0]
    n_times_test = y_test.shape[0]
    

In [ ]:
print(X_train.shape)
print(y_train.shape)

## 1.2 Classification

### The classifier

A classifier is an algorithm that aims to classify given inputs (here the power in given frequency bands) into outputs (here our 5 possible fingers + the idle class where the user does nothing). 

In this particular example we will use either the `LinearDiscriminantAnalysis` (imported here as `LDA`) from `sklearn.discriminant_analysis` or the `LinearSVC` from `sklearn.svm`. 
See their documentation here : 

https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html

https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC

In the sklearn API, all classes are using the `fit` function to train the algorithm and the `predict` function to get the predictions for the test set. 

### Fit the classifier

In [ ]:
estimator = LDA()

estimator.fit(X_train, y_train)

### Get the predicted labels

In [ ]:
y_pred = estimator.predict(X_test)
y_proba = estimator.predict_proba(X_test)

cue_pred = np.zeros([n_times_test, n_fingers+1])
for i in range(n_times_test):
    cue_pred[i, y_pred[i]] = 1

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)

cue_test = np.zeros([n_times_test, n_fingers+1])
for i in range(n_times_test):
    cue_test[i, y_test[i]] = 1

for i in range(n_fingers):
    ax[0].plot(cue_test[:, i])
    ax[1].plot(y_proba[:, i])
    ax[2].plot(cue_pred[:, i])
fig.suptitle('Finger trajectories (test set)')

## 1.3 Confusion matrix and metrics

Once you get your predictions you usually want to evaluate the performance of your decoder. In order to do so, a lot of different metrics have been developped with time. Most of the are using what is called a confusion matrix in order to be computed. 

### The confusion matrix

The confusion matrix represents how well our classifier is performing. The rows are the true labels we are expected to obtain and the columns are the labels our classifier actually gives us. 

In order to plot the confusion matrix, you can use the `from_predictions` function from the `ConfusionMatrixDisplay` class of `sklearn.metrics`.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html

In [ ]:
fig = plt.figure(figsize=(100,100))
disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred) # , text_kw={'fontsize': 10}

### The accuracy

A good way to consider how well the classifer is performing using the confusion matrix is the accuracy. The accuracy is equal to the total number of well predicted labels divided by the total number of labels. In the confusion matrix it is equal to the sum of the diagonal divided by the sum of all the elements of the matrix. 

You can get the accuracy using the `accuracy_score` function from `sklearn.metrics`. 

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score

In [ ]:
acc = accuracy_score(y_test, y_pred)
print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

### The balanced accuracy

In some cases there are more labels from some classes than others, for example here the finger 2 and 5 are less represented than others so their accuracy accounts for less than the others when we compute the accuracy metric. A good way to consider every labels with the same weight is to compute the balanced accuracy. It gives the accuracy for each class reweighted so that every class accounts for as much as the others. 

It is computed using the `balanced_accuracy_score` function from `sklearn.metrics`.

https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html#sklearn-metrics-balanced-accuracy-score

In [ ]:
for i in range(n_fingers+1):
    print("There are {} labels in test for class {}".format(np.sum(y_test==i), i+1))

plt.hist(y_test, bins=n_fingers+1, rwidth=0.5)


In [ ]:
acc = balanced_accuracy_score(y_test, y_pred)
print("Average score is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

## 1.4 The validation set

Sometimes algorithms and methods need to set a specific parameter to get better results. In order to do so, we usually use what we call a validation set. This set is usually a subdivision of the training set that works as a validation for the parameter we are trying to optimize. In the next example, we will try to find the best lag between the input and the output.

### Use the validation to compute a lag value

In [ ]:
lags = [i for i in range(-26, 26, 2)] 
X_train_val, X_val, y_train_val, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=13, shuffle=False)
lagged_accuracies = []
n_times_train_val = X_train_val.shape[0]
n_times_val = X_val.shape[0]
for i_lag, lag in enumerate(lags):
    X_train_lagged = X_train_val[max(-lag, 0):min(n_times_train_val-lag, n_times_train_val)]
    y_train_lagged = y_train_val[max(lag, 0):min(n_times_train_val+lag, n_times_train_val)]
    X_val_lagged = X_val[max(-lag, 0):min(n_times_val-lag, n_times_val)]
    y_val_lagged = y_val[max(lag, 0):min(n_times_val+lag, n_times_val)]
    
    estimator.fit(X_train_lagged, y_train_lagged)
    y_pred_val = estimator.predict(X_val_lagged)
    lagged_accuracies.append(accuracy_score(y_val_lagged, y_pred_val))
    print("Lag {}/{} ({:.2f}s) -> Accuracy = {:.2f}%".format(i_lag+1, len(lags), lag/25, lagged_accuracies[-1]*100))

### Compute the best lag

In [ ]:
plt.plot(np.linspace(-1, 1, len(lagged_accuracies)), lagged_accuracies)
best_lag = lags[np.argmax(lagged_accuracies)]
print("The best lag is {}. This lag correspond to {:.2f}s".format(best_lag, best_lag/25))

### Compute lagged X and y

In [ ]:
X_train_lagged = X_train[max(-best_lag, 0):min(n_times_train-best_lag, n_times_train)]
y_train_lagged = y_train[max(best_lag, 0):min(n_times_train+best_lag, n_times_train)]
n_times_train_lagged = n_times_train-abs(best_lag)

X_test_lagged = X_test[max(-best_lag, 0):min(n_times_test-best_lag, n_times_test)]
y_test_lagged = y_test[max(best_lag, 0):min(n_times_test+best_lag, n_times_test)]
n_times_test_lagged = n_times_test-abs(best_lag)

### Train the estimator and decode testing data with lagged estimates of X and y

In [ ]:
estimator.fit(X_train_lagged, y_train_lagged)
y_pred = estimator.predict(X_test_lagged)
y_proba = estimator.predict_proba(X_test_lagged)

cue_pred = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_pred[i, y_pred[i]] = 1

cue_test = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_test[i, y_test[i]] = 1

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)
for i in range(n_fingers):
    ax[0].plot(cue_test[:, i])
    ax[1].plot(y_proba[:, i])
    ax[2].plot(cue_pred[:, i])
fig.suptitle('Finger trajectories (test set)')

### Compute metrics 

In [ ]:
fig = plt.figure(figsize=(100,100))
disp = ConfusionMatrixDisplay.from_predictions(y_test_lagged, y_pred)

acc = accuracy_score(y_test_lagged, y_pred)
print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

acc = balanced_accuracy_score(y_test_lagged, y_pred)
print("Balanced accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

## 1.5 Feature selection and dimensionality reduction

In some cases in machine learning there are so much features that the model cannot correctly learn for the training data. In these cases, it is very common to compute dimensionnality reduction or spatial filters in order to keep only the features of interest for decoding. In this example we will use a Principal Components Analysis (the `PCA` class from `sklearn.decomposition`) and compute patterns for each finger in order to improve the results.

In `sklearn`, the classes that transform data without predicting are called transformers and use the functions `fit` and `transform` to process data. 

In [ ]:
X_train_pca = []
X_test_pca = []
for i in range(n_fingers):
    print("Computing PCA patterns for finger {}...".format(i+1))
    s_filt = PCA(60)
    s_filt.fit(X_train_lagged[y_train_lagged==i])
    X_train_pca.append(s_filt.transform(X_train_lagged))
    X_test_pca.append(s_filt.transform(X_test_lagged))

X_train_pca = np.concatenate(X_train_pca, axis=1)
X_test_pca = np.concatenate(X_test_pca, axis=1)

estimator.fit(X_train_pca, y_train_lagged)
y_pred_pca = estimator.predict(X_test_pca)
y_proba_pca = estimator.predict_proba(X_test_pca)

In [ ]:
cue_pred = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_pred[i, y_pred_pca[i]] = 1

cue_test = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_test[i, y_test[i]] = 1

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)
for i in range(n_fingers):
    ax[0].plot(cue_test[:, i])
    ax[1].plot(y_proba_pca[:, i])
    ax[2].plot(cue_pred[:, i])
fig.suptitle('Finger trajectories (test set)')

In [ ]:
fig = plt.figure(figsize=(100,100))
disp = ConfusionMatrixDisplay.from_predictions(y_test_lagged, y_pred_pca)


acc = accuracy_score(y_test_lagged, y_pred_pca)
print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

acc = balanced_accuracy_score(y_test_lagged, y_pred_pca)
print("Balanced accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

## 1.6 Get a more stable output (optional)

On decoders that are to be used by humans, it is important to get a more stable output so that it doesn't radically change every 40ms for example. This does not always improve classification but it can improve the feeling of control for the user with the side effect of adding a bit of lag. 

In this way, we can get the output probability of the decoder using the `predict_proba` function and smooth it using a convolution with a window. 

In [ ]:
win_size = 25
win = np.expand_dims(scipy.signal.windows.hamming(win_size), 1)
y_proba_stable = convolve(y_proba_pca, win, 'same')
y_pred_stable = np.argmax(y_proba_stable, axis=1)

cue_pred = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_pred[i, y_pred_stable[i]] = 1

cue_test = np.zeros([n_times_test_lagged, n_fingers+1])
for i in range(n_times_test_lagged):
    cue_test[i, y_test[i]] = 1

fig = plt.figure(figsize=(150,5))
ax = fig.subplots(3, 1)
for i in range(n_fingers):
    ax[0].plot(cue_test[:, i])
    ax[1].plot(y_proba_stable[:, i])
    ax[2].plot(cue_pred[:, i])
fig.suptitle('Finger trajectories (test set)')

### Get metrics

In [ ]:
fig = plt.figure(figsize=(100,100))
disp = ConfusionMatrixDisplay.from_predictions(y_test_lagged, y_pred_stable)

acc = accuracy_score(y_test_lagged, y_pred_stable)
print("Accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/np.unique(y_test).shape[0]))

acc = balanced_accuracy_score(y_test_lagged, y_pred_stable)
print("Balanced accuracy is {:.2f}%. Chance is at {:.2f}%".format(100*np.mean(acc), 100/6))

# Part 2 : Continuous decoding

In this part we will be interested in decoding precise hand movements of a monkey from ECOG data and in order to do so we will procede just as in the Part 1 with
- Pre-processing - where we will see artefact removal for example
- Feature extraction - by extracting time-frequency features on every electrodes using short term fourier transfrom
- Feature reduction using PCA

After this first part we will focus on classification paradigms with
- Cross validation in order to ensure the robustness of our testing procedure
- Regression using a linear model 

Finally we will evaluate the model performances using mean square error or correlation.

## 1.1 Data pre-processing

### Artefact removal 

### Feature extraction

### Feature selection and dimensionality reduction

## 1.2 Classification

### Train-test and cross-validation 

### Get the predicted labels

### Using the validation set

## 1.3 Metrics for continuous decoding

### RMSE

### Correlation